In [74]:
import numpy as np
import heapq
THINK = 1
REQ = 0
IDLE = 2
BUSY = 3
ARR = 4
DEP = 5
SWCH = 6
np.random.seed(42)

In [85]:
class Queue:
    def __init__(self):
        self.len = 1000
        self.que = np.empty(self.len, Request)
        self.front = -1
        self.rear = -1
        
    def increase_waittime(self, clock, tstamp):
        i = self.front
        while(i != self.rear):
            self.que[i].add_to_wait(clock, tstamp)
            i = (i+1)%self.len
        self.que[i].add_to_wait(clock, tstamp)
    
    def isFull(self):
        return True if((self.rear+1)%self.len == self.front) else False 
                    
    def isEmpty(self):
        return True if(self.front == -1) else False 
    
    def enqueue(self,req):
        if((self.rear+1)%self.len == self.front):
            print("Overflow")
            return False
        if(self.rear == -1):
            self.front += 1
            self.rear += 1
        else:
            self.rear = (self.rear+1)%self.len
        self.que[self.rear] = req
        return True
    
    def dequeue(self):
        if(self.front == -1):
            print("Underflow")
            return False
        req = self.que[self.front]
        if(self.rear == self.front):
            self.front = -1
            self.rear = -1
        else:
            self.front = (self.front+1)%self.len
        return req

In [86]:
class Request:
    def __init__ (self, user_id, t_req, tp_timeout, tp_service):
        self.user = user_id
        self.t_req = t_req
        self.tp_timeout = tp_timeout
        self.tp_service = tp_service
        self.tp_wait = 0
        self.core_num = -1
        
    def add_to_wait(self,clock,tstamp):
        self.tp_wait += (tstamp - clock)

    def assign_core(self, core_num):
        self.core_num = core_num

    def dec_time(self, sw_time):
        self.tp_service = self.tp_service - sw_time
        

In [98]:
class Simulation:
    def __init__(self):
        self.clock = 0.0
        self.no_of_users = 3
        self.user = []
        self.que = Queue()
        self.s = Server(1)
        for i in range(self.no_of_users):
            self.user.append(Users(i))  
        # time and user at next arrival    
        self.n_arrival, self.n_arr_u = self.generate_next_arrival()
        self.n_event = ARR
        self.n_depart = float('inf')
        self.event_pq = []
        heapq.heapify(self.event_pq)
    
    def generate_next_arrival(self):
        t = self.user[0].t_req 
        u = 0
        for i in range(self.no_of_users):
            if (self.user[i].t_req <  t):
                t = self.user[i].t_req
                u = i
        return t,u
    
    def advance_time(self):
        if(self.n_arrival < self.n_depart):
            self.n_event = ARR
            # checking any req waiting
            if not(self.que.isEmpty()):
                self.que.increase_waittime(self.clock,self.n_arrival) 
            # ask user to send request and put it in queue
            if not(self.user[self.n_arr_u].generate_request(self)):
                return False
            # ask server to serve if IDLE
            self.s.serve(self.n_arrival, self)
            self.clock = self.n_arrival
            self.n_arrival, self.n_arr_u = self.generate_next_arrival()
            if self.n_arrival < self.n_depart:
                self.n_event = ARR
            elif self.n_depart < self.n_arrival:
                self.n_event = DEP
            return
        
        if(self.n_depart < self.n_arrival):
            self.n_event = DEP
            n_depart = self.n_depart
            # ask server to depart the req and take new req from queue
            self.s.serve(self.n_depart,self)
            # checking any req waiting
            if(not(self.que.isEmpty())):
                self.que.increase_waittime(self.clock,self.n_depart)
            self.clock = n_depart
            self.n_arrival, self.n_arr_u = self.generate_next_arrival()
            if(self.n_arrival < self.n_depart):
                self.n_event = ARR
            elif(self.n_depart < self.n_arrival):
                self.n_event = DEP
            
    def display(self):
        print("===================================================")
        print("Clock :",self.clock)
        for i in range(self.no_of_users):
            print("User",i,":-")
            print("Request time :",self.user[i].t_req)
            print("Service time :",self.user[i].tp_service)
        print("------------------------------------------------------------------")    
        print("Next Arrival :",self.n_arrival,"Next Arriving User",self.n_arr_u)
        print("Next Departure:",self.n_depart)
        print("Next Event:",self.n_event)
        if(self.que.isEmpty()):
            print("Queue is Empty")
        else:
            print("Queue is not Empty")
        print("===================================================")
        

In [99]:
class Users:
    def __init__(self,user_id):
        self.id = user_id
        self.state = THINK
        self.req_dropped = 0
        self.req_timedout = 0
        self.waitt_list = []
        self.t_req = self.generate_thinktime()
        self.tp_timeout = self.generate_timeout()
        self.tp_service = self.generate_service()

    def generate_thinktime(self):
        return np.random.exponential(6)
    
    def generate_service(self):
        return np.random.exponential(0.048)
    
    def generate_timeout(self):
        return (1)
    
    def generate_request(self, sim):
        if (self.state == THINK):
            if(not(sim.que.isFull())):
                self.tp_timeout = self.generate_timeout()
                self.tp_service = self.generate_service()
                self.req = Request(self.id, self.t_req, self.tp_timeout, self.tp_service)
                sim.que.enqueue(self.req)
                self.state = REQ
                self.t_req = float('inf')
                return True
            else:
                self.req_dropped += 1
                self.t_req += self.generate_thinktime()
        return False
    
    def request_finish(self,tstamp):
        self.waitt_list.append(self.req.tp_wait)
        self.state = THINK
        self.t_req = tstamp + self.generate_thinktime()


In [104]:
class Event:
    def __init__(self, e_type, req, timestamp):
        self.e_type = e_type
        self.req = req
        self.timestamp = timestamp


    def __lt__(self, other):
        return self.timestamp < other.timestamp



In [ ]:
class Core:
    def __init__(self, sw_time):
        self.req = -1
        self.state = IDLE
        self.switch_time = sw_time

    def as_request(self, req):
        self.req = req
        self.state = BUSY

    def n_event(self, tstmp):
        if self.switch_time < self.req.tp_service:
            return Event(SWCH, self.req, tstmp + self.switch_time)
        else:
            return Event(DEP, self.req, tstmp + self.req.tp_service)

In [ ]:
class Server:
    def __init__(self,cores):
        self.state = IDLE
        self.num_sys = 0
        self.no_of_cores = cores
        self.cores_list = [Core(5) for _ in range(self.no_of_cores)]
        self.job_que = Queue()
        self.n_reqs = 0
        self.max_reqs = 10

        
    def serve(self,tstamp, sim, event):
        if event.e_type == DEP:
            print("---Serving DEP event---")
            sim.user[event.req].request_finish(tstamp)
            core = self.cores_list[event.req.core_num]
            if sim.que.isEmpty():
                core.state = IDLE
                print("Server idle.")
                # sim.n_depart = float('inf')
                return
            req = sim.que.dequeue()
            self.job_que.enqueue(req)
            ass_req = self.job_que.dequeue()
            core.as_request(ass_req)
            ass_req.assign_core(core)

            n_event = core.n_event(tstamp)
            heapq.heappush(sim.event_pq, n_event)
            # d_core = self.c_depart()
            # sim.n_depart_core = d_core
            # sim.n_depart = tstamp + self.cores_list[d_core].req.tp_service

        elif event.e_type == ARR:
            print("---Serving ARR event----")

            if self.n_reqs < self.max_reqs:
                req = sim.que.dequeue()
                self.job_que.enqueue(req)

                if self.n_reqs < self.no_of_cores:
                    for c in self.cores_list:
                        if c.state == IDLE:
                            print("Core idle.")
                            ass_req = self.job_que.dequeue()
                            c.as_request(self.job_que.dequeue())
                            ass_req.assign_core(c)

                            n_event = c.n_event(tstamp)
                            heapq.heappush(sim.event_pq, n_event)
                            break
            else:
                print("Server busy.")

        elif event.e_type == SWCH:
            print("---Serving SWITCH event----")
            core = self.cores_list[event.req.core_num]
            event.req.dec_time(core.switch_time)
            self.job_que.enqueue(event.req)

            ass_req = self.job_que.dequeue()
            core.as_request(ass_req)
            ass_req.assign_core(core)

            n_event = core.n_event(tstamp)
            heapq.heappush(sim.event_pq, n_event)






In [105]:
sim = Simulation()
sim.display()

sim.advance_time()
sim.display()

sim.advance_time()
sim.display()

Clock : 0.0
User 0 :-
Request time : 1.7967466610441165
Service time : 0.052140278331119054
User 1 :-
Request time : 2.241279489945742
Service time : 0.035237323003642976
User 2 :-
Request time : 4.747342787890785
Service time : 0.009810652776379476
------------------------------------------------------------------
Next Arrival : 1.7967466610441165 Next Arriving User 0
Next Departure: inf
Next Event: 4
Queue is Empty
---Serving ARR event----
Server idle.
Clock : 1.7967466610441165
User 0 :-
Request time : inf
Service time : 0.16765474237133887
User 1 :-
Request time : 2.241279489945742
Service time : 0.035237323003642976
User 2 :-
Request time : 4.747342787890785
Service time : 0.009810652776379476
------------------------------------------------------------------
Next Arrival : 2.241279489945742 Next Arriving User 1
Next Departure: 1.9644014034154553
Next Event: 5
Queue is Empty
---Serving DEP event---
Server idle.
Clock : 1.9644014034154553
User 0 :-
Request time : 10.9178736662443
S

In [106]:
sim.advance_time()
sim.display()

---Serving ARR event----
Server idle.
Clock : 2.241279489945742
User 0 :-
Request time : 10.9178736662443
Service time : 0.16765474237133887
User 1 :-
Request time : inf
Service time : 0.13464453212599595
User 2 :-
Request time : 4.747342787890785
Service time : 0.009810652776379476
------------------------------------------------------------------
Next Arrival : 4.747342787890785 Next Arriving User 2
Next Departure: 2.375924022071738
Next Event: 5
Queue is Empty


In [107]:
sim.advance_time()
sim.display()


---Serving DEP event---
Server idle.
Clock : 2.375924022071738
User 0 :-
Request time : 10.9178736662443
Service time : 0.16765474237133887
User 1 :-
Request time : 15.888836007276916
Service time : 0.13464453212599595
User 2 :-
Request time : 4.747342787890785
Service time : 0.009810652776379476
------------------------------------------------------------------
Next Arrival : 4.747342787890785 Next Arriving User 2
Next Departure: inf
Next Event: 4
Queue is Empty
